<a href="https://colab.research.google.com/github/changyuhsin1999/Nintendo_Switch_Game_Recommendation/blob/main/recommendation_by_genre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Nintendo_Switch_Game_Recommendation" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/Nintendo_Switch_Game_Recommendation.git'
!git clone "{git_path}"

Cloning into 'Nintendo_Switch_Game_Recommendation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 1.87 MiB | 9.05 MiB/s, done.


In [2]:
import os
import urllib
import zipfile

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
raw_df = pd.read_csv('/content/Nintendo_Switch_Game_Recommendation/switch-games.csv')
clean_df = raw_df.loc[:,['id','title','game_url','developer','genre','rating']].dropna()
final_df = clean_df[clean_df.rating != 0.0]
final_df.head()

,id,title,game_url,developer,genre,rating
3,e2f10932,Temtem,https://www.nintendolife.com/reviews/nintendo-...,Crema Games,Action Adventure RPG,7.8
5,4377dbf1,Splatoon 3,https://www.nintendolife.com/reviews/nintendo-...,Nintendo,Action,9.1
6,751a7bbf,JoJos Bizarre Adventure: All-Star Battle R,https://www.nintendolife.com/reviews/nintendo-...,CyberConnect2,Fighting,7.0
8,438bc630,Pac-Man World Re-PAC,https://www.nintendolife.com/reviews/nintendo-...,Bandai Namco,Action Adventure Platformer,6.9
9,7796d27e,Teenage Mutant Ninja Turtles: The Cowabunga Co...,https://www.nintendolife.com/reviews/nintendo-...,Digital Eclipse,Action Arcade Fighting Platformer,6.9


In [4]:
# create an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# apply the object to the genres column
tfidf_matrix = tfidf_vector.fit_transform(final_df['genre'])

In [33]:
from sklearn.metrics.pairwise import linear_kernel
# create the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)
sim_matrix.shape

(568, 568)

In [8]:
#Create reverse mapping of the game title so that we can eaily match back
indicies = pd.Series(final_df.index, index = final_df['title']).drop_duplicates()

In [29]:

def get_recommendations_by_genre(title, cosine_sim = sim_matrix):
  idx = indicies[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
  sim_scores = sim_scores[1:11]
  game_indicies = [i[0] for i in sim_scores]
  return final_df['title'].iloc[game_indicies]

In [30]:
get_recommendations_by_genre('Splatoon 3')

198             Mario Kart Live: Home Circuit
51                                 Chocobo GP
562                     10 Second Run Returns
103                              Cruisn Blast
303                            Race With Ryan
290                 Disney Tsum Tsum Festival
460                            Carnival Games
635                                1-2-Switch
178    Need for Speed: Hot Pursuit Remastered
299              Garfield Kart Furious Racing
Name: title, dtype: object

In [22]:
get_recommendations_by_genre('JoJos Bizarre Adventure: All-Star Battle R')

18                                           Live A Live
53     Atelier Sophie 2: The Alchemist of the Mysteri...
76                         BLUE REFLECTION: Second Light
85                                      Astria Ascending
136          Shin Megami Tensei III Nocturne HD Remaster
160                                   Bravely Default II
167      Atelier Ryza 2: Lost Legends & The Secret Fairy
223                                           Fairy Tail
241             Xenoblade Chronicles: Definitive Edition
273                Snack World: The Dungeon Crawl - Gold
Name: title, dtype: object